# Types composés: `struct` et `enum`


[Pierre-Antoine Champin](https://champin.net/) (W3C/Inria/Lyon 1)

http://github.com/pchampin/rust-envol-2025

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/2.0/fr/"><img alt="Contrat Creative Commons" style="border-width:0" src="http://i.creativecommons.org/l/by-nc-sa/2.0/fr/88x31.png" /></a>

# Types `struct`<a class="anchor" id="struct"></a>

Quelque part entre les `struct` du C et les `class` du C++

## Déclarer un type `struct`

In [2]:
struct Person {
    given_name: String,
    family_name: String,
    age: u8,
}

struct Color {
    r: u8,
    g: u8,
    b: u8,
}

In [3]:
// redéfinition du type `Color` avec une méthode `clone`,
// pour les besoins des exemples suivants
#[derive(Clone)]
struct Color {
    r: u8,
    g: u8,
    b: u8,
}

## Initialiser un `struct`

In [4]:
let black = Color { b: 0, r: 0, g: 0 };

let blue = Color { b: 255, ..black };

let r = 12;
let g = 34;
let b = 56;
let mut mycolor = Color { r, g, b }; // équivalent à Color { r: r, g: g, b: b }

## Accéder aux champs d'un `struct`

In [5]:
// RAPPEL: 'mycolor' a pour type 'struct Color { r: u8, g: u8, b: u8 }'

// en lecture
let x = mycolor.b;

// en écriture
mycolor.r = x+1;

## `struct` « tuple »

In [6]:
struct Rgb(u8, u8, u8);

let green = Rgb(0, 255, 0);

// les champs sont accédés comme ceux d'un tuple (.0, .1, ...)
let x = green.1;

## `struct` singleton

In [7]:
struct Foo;

let f = Foo;

NB: ces types n'ont qu'une seule valeur (notée de manière identique au type).

Cette valeur a une taille mémoire nulle.

# Affectation déstructuranre (a.k.a. *pattern matching*) 🦀 <a class="anchor" id="destructuring"></a>


Au lieu d'écrire

In [8]:
// RAPPEL: 'mycolor' a pour type 'struct Color { r: u8, g: u8, b: u8 }'

let r = mycolor.r;
let g = mycolor.g;
let b = mycolor.b;

on peut écrire

In [9]:
let Color { r, g, b } = mycolor;

Au lieu d'écrire

In [10]:
// RAPPEL: 'mycolor' a pour type 'struct Color { r: u8, g: u8, b: u8 }'

let r1 = mycolor.r;
let g1 = mycolor.g;
let b1 = mycolor.b;

on peut écrire

In [11]:
let Color { r: r1, g: g1, b: b1 } = mycolor;

Au lieu d'écrire

In [12]:
// RAPPEL: 'green' a pour type 'struct Rgb (u8, u8, u8)'

let r2 = green.0;
let g2 = green.1;
let b2 = green.2;

on peut écrire

In [13]:
let Rgb(r2, g2, b2) = green;

## Affectation déstructurante partielle

Au lieu d'écrire

In [14]:
// RAPPEL: 'mycolor' a pour type 'struct Color { r: u8, g: u8, b: u8 }'

let r  = mycolor.r;
let b1 = mycolor.b;

on peut écrire

In [15]:
let Color { b: b1, r, .. } = mycolor;

Au lieu d'écrire

In [16]:
// RAPPEL: 'green' a pour type 'struct Rgb(u8, u8, u8)'

let r  = green.0;
let b1 = green.2;

on peut écrire

In [17]:
let Rgb(r, _, b1) = green;

* `_` permet d'ignorer un champ positionnel
* `..` permet d'ignorer plusieurs champs (postionnels ou nommés)

## Affectation déstructurante de tuples

In [18]:
let t = (1, 0.5, "txt");

let (a, _, c) = t;

L'affectation multiple de variables en est un cas particulier:

In [19]:
let (x, y) = (a-1, a+1);

## Affectation déstructurante de tableaux/tranches

In [20]:
let t = [11, 22, 33, 44, 55];

let [a, b, c, d, e] = t;
let [_, second, .., last] = t;
println!("{second} {last}");

22 55


## Affectation déstructurante récursive

In [21]:
struct Gradient(Color, Color);
let gr = Gradient(black.clone(), blue.clone());

let Gradient(Color { b: b1, .. }, Color { b: b2, .. }) = gr;
println!("{b1} {b2}");

0 255


## Affectation déstructurante et déplacement

Comme toute affectation en Rust,
l'affectation déstructurante a une sémantique de "déplacement"

– sauf pour les types implémentant le trait `Copy`,
ce qui est le cas de tous les exemples précédents (champs de type `u8`).

In [22]:
let gr = Gradient(black.clone(), blue.clone());
let Gradient(c1, c2) = gr;
//println!("{}", gr.0.b); // NE COMPILE PAS

La variable `g` est maintenant inutilisable,
les valeurs de ses champs ont été *déplacés* dans `c1` et `c2`.

## Déstructurer une référence

In [23]:
let mut gr = Gradient(black.clone(), blue.clone());
{
    let Gradient(c3, c4) = &gr;
    // c3 et c4 sont de type &Color
    println!("A. {} {}", c4.b, gr.1.b);
}
{ 
    let Gradient(c5, c6) = &mut gr;
    // c5 et c6 sont de type &mut Color
    // gr.1.b = 128; // NE COMPILE PAS
    c6.b = 128;
};
println!("B. {}", gr.1.b);


A. 255 255
B. 128


# Types `enum` 🦀 <a class="anchor" id="enum"></a>

Un mélange des `enum` et des `union` du C.

## Déclarer un type `enum` simple

In [24]:
enum Beatle {
    George,
    John,
    Paul,
    Ringo,
}

`George`, `John`... sont appelées les **variantes** du type `Beatle`.

## Initialiser un `enum` simple

In [25]:
let b1 = Beatle::John;

In [26]:
use Beatle::*;
let b2 = Paul;
let b3 = George;

## Utiliser un `enum` simple

In [27]:
use Beatle::*;

fn instrument(b: &Beatle) -> &'static str {
    match b {
        George | John => "guitar",
        Paul => "bass",
        Ringo => "drums",
    }
}

## Déclarer un type `enum` complexe

In [28]:
struct Point { x: f64, y: f64 }

enum Figure2D {
    Plane,
    Line(Point, Point),
    Circle { centre: Point, radius: f64 },
}

## Initialiser un `enum` complexe

In [29]:
let p = Figure2D::Plane;
let l = Figure2D::Line(Point { x: 1.0, y: 2.0 }, Point { x: 3.0, y: 4.0 });

let centre = Point { x: 5.0, y: 6.0 };
let c = Figure2D::Circle{ radius: 7.0, centre };

// p, l et c ont toutes le même type: Figure2D
let figures = [p, l, c]; // a pour type [Figure2D; 3]

## Utiliser `match` avec un `enum` complexe

In [30]:
use std::f64::consts::PI;
use Figure2D::*;

fn area(f: Figure2D) -> f64 {
    match f {
        Plane => f64::INFINITY,
        Line(_, _) => 0.0,
        Circle { radius: r, .. } => PI*r*r,
    }
}

## Clause `if let`

Au lieu d'écrire:

In [31]:
match &figures[0] {
    Line(p1, p2) => { /* faire quelque chose avec p1 and p2 */ }
    _ => {}
};

on peut écrire, de manière plus concise:

In [32]:
if let Line(p1, p2) = &figures[0] {
    // fait quelque chose avec p1 et p2
};

* on peut aussi utiliser `else` avec `if let`
* il existe aussi une clause `while let`

# Méthodes  <a class="anchor" id="methods"></a>

## Tous les types peuvent avoir des méthodes en Rust 🦀

... même les types primitifs.

E.g.

In [33]:
let x = "hello".len();
let y = 1000.max(x);
let z = PI.log2();

## Définir les méthodes d'un nouveau type

In [34]:
impl Color {
    /// Méthode de lecture-seule
    fn saturation(&self) -> f64 {
        //        ^^^^^ emprunt immutable de self
        let cmax = self.r.max(self.g).max(self.b);
        if cmax == 0 {
            0.0
        } else {
            let cmax = cmax as f64;
            let cmin = self.r.min(self.g).min(self.b) as f64;
            (cmax - cmin) / cmax
        }
    }
    
    /// Méthode modifiant la valeur
    fn lighten(&mut self, delta: u8) {
        //     ^^^^^^^^^ emprunt mutable de self
        self.r = self.r.saturating_add(delta);
        self.g = self.g.saturating_add(delta);
        self.b = self.b.saturating_add(delta);
    }
}

In [35]:
let mut c = Color { r: 180, g: 0, b: 0};
println!("A. {}", c.saturation());
c.lighten(127);
println!("B. {}", c.saturation());

A. 1
B. 0.5019607843137255


## Autres méthodes, fonctions associées

In [36]:
impl Color { // Il peut y avoir plusieurs blocs 'impl'

    /// Méthode consommant la valeur
    fn to_rgb(self) -> Rgb {
        //    ^^^^ déplacement de self dans le corps de la méthode
        let Color { r, g, b } = self;
        Rgb(r, g, b)
    }
    
    /// Fonction associée, sans paramètre `self`,
    /// appelée directement sur le type.
    /// Pattern commun pour définir un "constructuer"
    fn new(r: u8, g: u8, b: u8) -> Color {
        Color { r, g, b }
    }
}

In [37]:
let navy = Color::new(0, 0, 128);
println!("A. {}", navy.saturation());
let x: Rgb = navy.to_rgb(); 
// println!("{}", navy.saturation()); // does not compile
println!("B. {} {} {}", x.0, x.1, x.2);

A. 1
B. 0 0 128


# `enums` standards 🦀  <a class="anchor" id="std-enums"></a>

## Le type `Option<T>` 🦀

### Motivation

* En Java, une fonction retournant `MyObject` peut retourner une instance de `MyObject`... *ou* `null`. (Même chose en C avec les fonctions retournant un pointeur)

* C'est une entorse aux règles de typage, car `null` n'est *pas* une instance de `MyObject`

  + toute tentative de l'utiliser comme un `MuObject` résultera dans une erreur (`NullPointerException`)


* En Rust, une fonction retournant le type `T` **doit** retourner une valeur de type `T`.

* Mais il y a des cas ou on souhaite retourner `un `T` ou rien du tout`. Exemples :

  + la méthode `pop()` de `Vec<T>` retire et retourne le dernier élément du `Vec` s'il en contient un, mais ne faite rien (et ne *retourne* rien) sinon
  
  + la méthode `get(i)` d'un tableau `[T]` retourne l'élément à l'indice *i* s'il existe, ou rien si *i* dépasse la taille du tableau


* Le type de retour de ces finctions est `Option<T>`, un `enum` à deux variantes:

  + `None`, représentant l'absence de valeur
  + `Some(t)`, où *t* est une valeur de `T`

### Exemple d'utilisation de `Option<T>`

In [38]:
let mut a: Vec<i32> = vec![11, 22, 33, 44];

println!("{a:?}");
if let Some(i) = a.pop() {
    println!("retiré la valeur {i}");
} else {
    println!("rien à retirer");
}
println!("{a:?}");

[11, 22, 33, 44]
retiré la valeur 44
[11, 22, 33]


### La méthode `unwrap`

Lorsqu'on a la certitude qu'une option n'est pas `None`,
on peut utiliser la méthode `unwrap` pour récupérer sa valeur.

In [39]:
let mut a: Vec<i32> = vec![11, 22, 33, 44];

println!("{a:?}");
if a.len() >= 2 {
    let x = a.pop().unwrap();
    let y = a.pop().unwrap();
    println!("retiré les valeurs {x} et {y}");
} else {
    println!("pas assez de valeurs");
}
println!("{a:?}");

[11, 22, 33, 44]
retiré les valeurs 44 et 33
[11, 22]


Si l'on s'est trompé et qu'on appelle `unwrap` sur `None`,
une erreur (*panic*) est déclenchée.

## Le type `Result<T,E>` 🦀

### Motivation

* Certaines fonctions peuvent réussir (et retourner leur résultat normalement) ou *échouer*

  + Dans le 2nd cas, on peut souhaiter connaître la raison de l'échec
  
* En Rust, ces fonctions retournent un `Result<T,E>`, un `enum` avec les variantes suivantes:

  + `Ok(t)` où *t* est une valeur de type `T`, représentant le résultat normal de la fonction
  + `Err(e)` où *e* est une valeur de type `E`, indiquant un échec

### `Result<T, E>` in action

In [40]:
use std::fs::read

match read("README.md") {
    Ok(content) => println!("Content: {} bytes", content.len()),
    Err(e) => println!("ERROR: {}", e),
};

Content: 740 bytes


NB: le type `Result<T,E>` possède également une méthode `unwrap`,
dans le cas où l'on sait qu'on récupère un `Ok`
(ou qu'on ne souhaite pas gérer l'erreur mieux qu'en interrompant le programme).

In [41]:
let content = read("README.md").unwrap();
println!("{} bytes", content.len());

740 bytes


### Relayer les erreurs

In [42]:
use std::io;
use std::fs::{read, write};

/// Copie au plus 42 octets d'un fichier dans un autre,
/// et retourne le nombre d'octets copiés.
fn copy_start(filename1: &str, filename2: &str) -> Result<usize, io::Error> {
    let mut content = match read(filename1) {
        Ok(c) => c,
        Err(e) => return Err(e),
    };
    content.truncate(42);
    match write(filename2, &content) {
        Ok(()) => {},
        Err(e) => return Err(e),
    };
    Ok(content.len())
}

* avantage: `Result` nous oblige a gérer explicitement les erreurs
* inconvénient: la gestion d'erreur est mélée au flot normal d'exécution, nuisant à la lisibilité du code
* NB: tous ces `match` ont fondamentalement la même structure

### Relayer les erreurs à la manière Rust 🦀

In [43]:
use std::io;
use std::fs::{read, write};

/// Copie au plus 42 octets d'un fichier dans un autre,
/// et retourne le nombre d'octets copiés.
fn copy_start(filename1: &str, filename2: &str) -> Result<usize, io::Error> {
    let mut content = read(filename1)?;
    content.truncate(42);
    write(filename2, &content)?;
    Ok(content.len())
}

L'opérateur `?` est un raccourci pour les `match` récurents de la version précédente.

Il suppose que la fonction retourne un `Result` avec le même type d'erreur (ou un type compatible).

Ressemble à la gestion des *exceptions* dans d'autres langages, mais de manière plus explicite.